In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import cv2
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array, smart_resize
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.preprocessing import image

In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df.head()

In [ ]:
class_name = df['labels'].value_counts().index
class_count = df['labels'].value_counts().values
df['labels'] = df['labels'].astype('category')
df['label_num'] = df['labels'].cat.codes

In [ ]:
submission = pd.read_csv("../input/plant-pathology-2021-fgvc8/sample_submission.csv")
submission.head()

In [ ]:
test_dir = '../input/plant-pathology-2021-fgvc8/test_images'
pred = []
model = models.load_model('../input/trainmodel/best_model.h5')

In [ ]:
for image in os.listdir(test_dir):
    path = os.path.join(test_dir, image)
    img = load_img(path)
    img = img_to_array(img)
    img = smart_resize(img, (150,150))
    img = tf.reshape(img, (-1, 150, 150, 3))
    temp = model.predict(img/255.)
    temp = np.argmax(temp)
    pred = np.append(pred,temp)

In [ ]:
submission_result = pd.DataFrame({'image' : submission.image, 'labels' : pred})
submission_result['labels'] = submission_result['labels'].astype(int)
class_map = dict(sorted(df[['label_num', 'labels']].values.tolist()))
submission_result['labels'] = submission_result['labels'].map(class_map)
submission_result.to_csv('submission.csv', index=False)

In [ ]:
print("Competetion Complete!!")